---
title: "Python fundamentals for data wrangling"
author: "Tobias Gerstenberg, adapted by Bria Long, Python version by Eshin Jolly"
date: "January 10th, 2022, adapted Sept, 2025"
# jupyter: fundamentals-workshop
# execute: 
#   cache: true
---

In [ ]:
#| eval: false
#| include: false
# Package installation handled by pixi environment

# Data wrangling 1

This notebook takes a look at how to wrangle data using the [Polars](https://pola.rs/) library. The nice thing about Python is that it has a powerful ecosystem for data science, with Polars providing blazingly fast data manipulation capabilities. Many analysis pipelines use both Python and R. However, Python excels at data wrangling and analysis with its rich scientific computing ecosystem, making it particularly popular in experimental psychology and data science.

## Learning goals

- Review Python basics (incl. data types, data structures, operators, control flow, and functions).
- Learn how method chaining works in Polars (equivalent to R's pipe `%>%` operator).
- See different ways for getting a sense of one's data.
- Master key data manipulation methods from the `polars` library (incl. `filter()`, `sort()`, `rename()`, `select()`, `with_columns()`) as well as helper functions and selectors.

## Load packages

Let's first load the packages that we need for this chapter.

In [ ]:
import polars as pl                    # for data wrangling
from polars import when, lit, col as c # shorter syntax helpers
import seaborn as sns                  # for statistical visualization
import matplotlib.pyplot as plt       # for basic plotting
import numpy as np                     # for numerical operations

# Configure display options
pl.Config.set_tbl_rows(10)

## Some Python basics

To test your knowledge of Python basics, I recommend taking a free interactive tutorial like [Python for Everybody](https://www.py4e.com/) or the [Python tutorial on DataCamp](https://www.datacamp.com/courses/intro-to-python-for-data-science). Here, I will just give a very quick overview of some of the basics.

### Data types

Variables in Python can have different data types. @tbl-data-types shows the most common ones.

In [ ]:
#| label: tbl-data-types
#| tbl-cap: "Most commonly used data types in Python."
#| echo: false

import pandas as pd
data_types_df = pd.DataFrame({
    'Type': ['int', 'float', 'str', 'bool', 'None'],
    'Example': ['`1`, `3`, `48`', '`1.0`, `3.14`, `48.5`', '`"Steve"`, `"a"`, `"78"`', '`True`, `False`', '`None`']
})

print(data_types_df.to_string(index=False))

For strings you can use either `"` or `'`. Python has built-in functions to convert a variable from one type to another. `None` is used for missing values.

In [ ]:
tmp1 = "1"  # we start with a string
print(f"tmp1 type: {type(tmp1)}")

tmp2 = int(tmp1)  # turn it into an integer
print(f"tmp2 type: {type(tmp2)}")

tmp3 = float(tmp2)  # turn that into a float
print(f"tmp3 type: {type(tmp3)}")

tmp4 = str(tmp3)  # and go full cycle by turning it back into a string
print(f"tmp4 type: {type(tmp4)}")

print(f"tmp1 == tmp4: {tmp1 == tmp4}")  # checks whether tmp1 and tmp4 are the same

The `type()` function shows us the data type of a Python object.

### Data structures

Python has several different data structures. @tbl-data-structures shows the ones you're most likely to come across:

In [ ]:
#| label: tbl-data-structures
#| tbl-cap: "Most commonly used data structures in Python."
#| echo: false

structures_df = pd.DataFrame({
    'Structure': ['list', 'tuple', 'dictionary', 'set', 'DataFrame'],
    'Description': [
        'ordered, mutable collection of items',
        'ordered, immutable collection of items',
        'unordered collection of key-value pairs',
        'unordered collection of unique items',
        'tabular data structure with labeled columns'
    ]
})

print(structures_df.to_string(index=False))

#### Lists

We build lists using square brackets `[]`, and we use indexing `[]` to access one or more elements of a list.

In [ ]:
numbers = [1, 4, 5]  # make a list
print(numbers[1])    # access the second element (0-indexed!)
print(numbers[0:2])  # access the first two elements
print(numbers[::2])  # access every other element

In Python (like many programming languages), indexing starts at 0, so `numbers[0]` refers to the first element.

#### Dictionaries

We build dictionaries using curly braces `{}` with key-value pairs, and we use keys to access values.

In [ ]:
person = {
    'name': 'Luke',
    'age': 23,
    'profession': 'Jedi'
}

print(person['name'])  # access value by key
print(person.get('age'))  # alternative way to access
print(list(person.keys()))  # get all keys

#### DataFrames

In [ ]:
df = pl.DataFrame({
    'participant_id': [1, 2, 3],
    'participant_name': ['Leia', 'Luke', 'Darth']
})

print(df)  # the complete DataFrame
print(df['participant_id'])  # all participant IDs
print(df['participant_name'][1])  # name of the second participant

We'll use DataFrames a lot. DataFrames are like a table with column names. DataFrames are flexible in that different columns can have different data types. For example, one column might be strings, another integers, and another floats.

Here we used the `pl.DataFrame()` function to create the DataFrame. Polars DataFrames are optimized for performance and provide a rich API for data manipulation.

### Operators

@tbl-logical-operators shows the comparison operators that result in boolean outputs.

In [ ]:
#| label: tbl-logical-operators
#| tbl-cap: "Table of comparison operators that result in boolean (True/False) outputs."
#| echo: false

operators_df = pd.DataFrame({
    'Symbol': ['`==`', '`!=`', '`>`, `<`', '`>=`, `<=`', '`and`, `or`, `not`', '`in`'],
    'Name': ['equal to', 'not equal to', 'greater/less than',
             'greater/less than or equal', 'logical operators: and, or, not',
             'checks whether an element is in an object']
})

print(operators_df.to_string(index=False))

The `in` operator is very useful, and we can use it like so:

In [ ]:
x = [1, 2, 3]
print(2 in x)
print([3, 4] in x)  # Note: this checks if the list [3, 4] is in x, not the elements
print(all(item in x for item in [1, 3]))  # Check if all elements are in x

It's particularly useful for filtering data as we will see below.

### Control flow

#### if-then-else

In [ ]:
number = 3

if number == 1:
    print("The number is 1.")
elif number == 2:
    print("The number is 2.")
else:
    print("The number is neither 1 nor 2.")

We can also use conditional expressions (ternary operator) as a shorthand:

In [ ]:
number = 3
result = "correct" if number == 1 else "false"
print(result)

#### for loop

In [ ]:
sequence = range(1, 11)  # equivalent to 1:10 in R

for i in sequence:
    print(i)

#### while loop

In [ ]:
number = 1

while number <= 10:
    print(number)
    number = number + 1

### Functions

In [ ]:
def add_two_numbers(a, b):
    result = a + b
    return f"The result is {result}"

print(add_two_numbers(1, 2))

I've used an f-string here to format the string with the number. Python functions can return multiple objects using tuples, and can have default arguments and keyword arguments.

#### Some often used functions

In [ ]:
#| label: tbl-common-functions
#| tbl-cap: "Some frequently used functions."
#| echo: false

functions_df = pd.DataFrame({
    'Function': ['`len()`', '`type()`', '`range()`', '`sum()`', '`max()`', '`min()`',
                 '`sorted()`', '`enumerate()`', '`zip()`'],
    'Description': ['length of an object', 'type of an object', 'generate a sequence of numbers',
                   'sum of elements', 'maximum', 'minimum', 'return sorted list',
                   'enumerate with indices', 'zip multiple iterables']
})

print(functions_df.to_string(index=False))

### Method chaining in Polars

In [ ]:
#| out-width: "80%"
#| echo: false
#| fig-cap: "Method chaining allows us to chain operations together in a readable way."

# Note: In a real implementation, you'd want to include an actual image here
# For now, we'll just have a text placeholder
print("🔗 Method chaining in Polars works similarly to R's pipe operator")

Method chaining is a powerful feature in Polars that allows us to chain several operations together in a natural, readable order. This is similar to R's pipe operator `%>%`.

Let's consider an example of data processing (this code block is conceptual):

In [ ]:
#| eval: false

# Traditional nested approach (harder to read)
result = df_final(
    df_sort(
        df_filter(
            df_select(df_original, ["col_a", "col_b"]),
            condition
        ),
        "col_a"
    )
)

# Method chaining approach (easier to read)
result = (df_original
    .select(["col_a", "col_b"])
    .filter(condition)
    .sort("col_a")
)

This method chaining approach is much easier to read and write, since it represents the order in which we want to do things!

Abstractly, method chaining does the following:

> `df.method1().method2()` applies method1 to df, then method2 to the result

For example, we could chain operations like:

In [ ]:
# Create a simple example
sample_data = pl.DataFrame({
    "x": [1, 2, 3, 4],
    "y": [10, 20, 30, 40]
})

# Method chaining example
result = (sample_data
    .filter(c("x") > 2)
    .select("y")
    .sum()
)

print(result)

Method chaining allows us to take step-by-step actions in a way that fits the logical ordering of how we want to transform our data.

> **Tip**: When writing method chains, put each method on a new line and wrap the entire chain in parentheses. This makes the code much more readable!

A key advantage of using method chaining is that you don't have to save intermediate computations as new variables, which helps keep your workspace clean!

#### Practice 1

Let's practice method chaining.

In [ ]:
# here are some numbers
x = [1, 2, 3, 4, 5]

# taking the sum the traditional way
print(sum(x))

# now create a DataFrame and use method chaining (write your code underneath)

In [ ]:
# some more numbers in a DataFrame
df_practice = pl.DataFrame({"values": [10, 9, 8, 7, 6, 5]})

# the traditional way
import math
result = math.sqrt(df_practice.select(pl.mean("values")).item())
print(result)

# the method chaining way (write your code underneath)

## A quick note on naming things

Personally, I like to name things in a (pretty) consistent way so that I have no trouble finding stuff even when I open up a project that I haven't worked on for a while. I try to use the following naming conventions:

In [ ]:
#| echo: false
#| label: tbl-naming-conventions
#| tbl-cap: "Some naming conventions I adopt to make my life easier."

naming_df = pd.DataFrame({
    'Name': ['df_thing', 'list_thing', 'func_thing', 'tmp_thing'],
    'Use': ['for DataFrames', 'for lists/arrays', 'for functions', 'for temporary variables']
})

print(naming_df.to_string(index=False))

## Looking at data

Polars doesn't include built-in datasets like R's tidyverse, so let's create a dataset similar to the starwars data for our examples.

In [ ]:
# Create a starwars-like dataset
df_starwars = pl.DataFrame({
    'name': ['Luke Skywalker', 'C-3PO', 'R2-D2', 'Darth Vader', 'Leia Organa',
             'Owen Lars', 'Beru Whitesun lars', 'R5-D4', 'Biggs Darklighter', 'Obi-Wan Kenobi'],
    'height': [172, 167, 96, 202, 150, 178, 165, 97, 183, 182],
    'mass': [77.0, 75.0, 32.0, 136.0, 49.0, 120.0, 75.0, 32.0, 84.0, 77.0],
    'hair_color': ['blond', None, None, None, 'brown', 'brown,grey', 'brown', None, 'black', 'auburn,white'],
    'skin_color': ['fair', 'gold', 'white,blue', 'white', 'light', 'light', 'light', 'white,red', 'light', 'fair'],
    'eye_color': ['blue', 'yellow', 'red', 'yellow', 'brown', 'blue', 'blue', 'red', 'brown', 'blue-gray'],
    'birth_year': [19.0, 112.0, 33.0, 41.9, 19.0, 52.0, 47.0, None, 24.0, 57.0],
    'sex': ['male', 'none', 'none', 'male', 'female', 'male', 'female', 'none', 'male', 'male'],
    'gender': ['masculine', 'masculine', 'masculine', 'masculine', 'feminine', 'masculine', 'feminine', 'masculine', 'masculine', 'masculine'],
    'homeworld': ['Tatooine', 'Tatooine', 'Naboo', 'Tatooine', 'Alderaan', 'Tatooine', 'Tatooine', 'Tatooine', 'Tatooine', 'Stewjon'],
    'species': ['Human', 'Droid', 'Droid', 'Human', 'Human', 'Human', 'Human', 'Droid', 'Human', 'Human']
})

Before manipulating the data, it's often useful to take a quick direct look at the data.

There are several ways of taking a look at data in Python/Polars. You can view DataFrames directly in Jupyter notebooks, or use various methods to inspect the data in the console.

Here are some useful methods:

### `head()`

Without any extra arguments specified, `head()` shows the top five rows of the data by default.

In [ ]:
df_starwars.head()

### `describe()`

`describe()` provides summary statistics for numeric columns, similar to R's `summary()`.

In [ ]:
df_starwars.describe()

### `schema`

The `schema` property shows the column names and their data types.

In [ ]:
df_starwars.schema

### `unique()`

`unique()` shows all the unique values for a column.

In [ ]:
df_starwars.select(c("species").unique())

### `value_counts()`

`value_counts()` shows a count of all the different unique values in a column.

In [ ]:
df_starwars.select(c("eye_color").value_counts())

It's possible to do grouped counts by combining several variables.

In [ ]:
# df_starwars.select(c("eye_color", "gender").value_counts()).head(10)

### Other tools for taking a quick look at data

#### Basic info about the DataFrame

In [ ]:
print(f"Shape: {df_starwars.shape}")
print(f"Columns: {df_starwars.columns}")
print(f"Data types: {df_starwars.dtypes}")

#### Checking for missing values

In [ ]:
df_starwars.null_count()

Once we've taken a look at the data, the next step would be to visualize relationships between variables of interest using seaborn.

In [ ]:
# Quick visualization example
plt.figure(figsize=(8, 6))
df_pandas = df_starwars.to_pandas()  # Convert to pandas for seaborn
sns.scatterplot(data=df_pandas, x='height', y='mass', hue='species')
plt.title('Height vs Mass by Species')
plt.show()

## Wrangling data

We use the methods in the `polars` library to manipulate our data.

### `filter()`

`filter()` lets us apply logical (and other) operators (see @tbl-logical-operators) to subset the data. Here, I've filtered out the masculine characters.

In [ ]:
df_starwars.filter(c("gender") == "masculine")

We can combine multiple conditions in the same call. Here, I've filtered out masculine characters, whose height is greater than the median height (i.e. they are in the top 50 percentile), and whose mass was not null.

In [ ]:
df_starwars.filter(
    (c("gender") == "masculine") &
    (c("height") > df_starwars.select(pl.median("height")).item()) &
    (c("mass").is_not_null())
)

Many aggregation functions like `mean()`, `median()`, `var()`, `std()`, `sum()` in Polars handle null values automatically by default, which is different from some other libraries.

You can use `&` (and), `|` (or), and `~` (not) to combine logical operations. Make sure to use parentheses when combining several logical operators to indicate which logical operation should be performed first:

In [ ]:
df_starwars.filter(
    ((c("skin_color").str.contains("dark|pale")) | (c("sex") == "none")) &
    (c("height") > 170)
)

This filters for starwars characters that have either "dark" or "pale" in their skin color, or whose sex is "none", and whose height is at least 170 cm. The `str.contains()` method with a regex pattern is useful when there are multiple options.

### `sort()`

`sort()` allows us to sort the values in a DataFrame by one or more column entries.

In [ ]:
df_starwars.sort(["hair_color", "height"], descending=[False, True])

Here, I've sorted the DataFrame first by `hair_color` (ascending), and then by `height` (descending).

### `rename()`

`rename()` renames column names using a dictionary mapping.

In [ ]:
df_starwars.rename({"name": "person", "mass": "mass_kg"})

The new variable names are the values in the dictionary, and the old names are the keys.

### `select()`

`select()` allows us to select a subset of the columns in the DataFrame.

In [ ]:
df_starwars.select(["name", "height", "mass"])

We can select multiple columns using slicing or various helper functions:

In [ ]:
df_starwars.select(pl.col("name", "height", "mass", "birth_year"))

Or select columns by data type:

In [ ]:
df_starwars.select(pl.col(pl.Float64, pl.Int64))  # Select all numeric columns

We can also *deselect* columns:

In [ ]:
df_starwars.select(pl.exclude(["name", "birth_year"]))

And select columns by pattern matching:

In [ ]:
df_starwars.select(pl.col("^.*_color$"))  # every column that ends with "_color"

We can rename some of the columns using `select()` like so:

In [ ]:
df_starwars.select([
    pl.col("name").alias("person"),
    "height",
    pl.col("mass").alias("mass_kg")
])

### Practice 2

Create a DataFrame that:
- only has the species `Human` and `Droid`
- with the following data columns (in this order): name, species, birth_year, homeworld
- is arranged according to birth year (with the lowest entry at the top of the DataFrame)
- and has the `name` column renamed to `person`

In [ ]:
# write your code here

### `with_columns()`

`with_columns()` is used to change existing columns or make new ones.

In [ ]:
(df_starwars
    .with_columns([
        (c("height") / 100).alias("height_m"),  # to get height in meters
        (c("mass") / (c("height") / 100)**2).alias("bmi")  # bmi = kg / (m^2)
    ])
    .select(["name", "height_m", "mass", "bmi"])
)

Here, I've calculated the BMI for the different starwars characters. I first created a height in meters variable, and then created the new column "bmi".

A useful function for `with_columns()` is `when().then().otherwise()` which is equivalent to R's `ifelse()`. Here is an example:

In [ ]:
(df_starwars
    .with_columns(
        when(c("height") > df_starwars.select(pl.median("height")).item())
        .then(lit("tall"))
        .otherwise(lit("short"))
        .alias("height_categorical")
    )
    .select(["name", "height", "height_categorical"])
)

`when().then().otherwise()` works by first specifying the condition, then what should be returned if the condition is true, and finally what should be returned otherwise.

#### Working with multiple columns

Polars provides powerful ways to apply operations to multiple columns at once using selectors and expressions.

For example, let's imagine that we want to standardize (z-score) several variables in our DataFrame:

In [ ]:
(df_starwars
    .with_columns([
        ((c("height") - c("height").mean()) / c("height").std()).alias("height_z"),
        ((c("mass") - c("mass").mean()) / c("mass").std()).alias("mass_z"),
        ((c("birth_year") - c("birth_year").mean()) / c("birth_year").std()).alias("birth_year_z")
    ])
    .select(["name", "height_z", "mass_z", "birth_year_z"])
)

We can also apply functions to multiple columns more efficiently:

In [ ]:
numeric_cols = ["height", "mass", "birth_year"]

(df_starwars
    .with_columns([
        ((c(col) - c(col).mean()) / c(col).std()).alias(f"{col}_z")
        for col in numeric_cols
    ])
    .select(["name"] + [f"{col}_z" for col in numeric_cols])
)

Sometimes, you want to apply a function only to those columns that have a particular data type:

In [ ]:
(df_starwars
    .with_columns(
        pl.col(pl.Float64, pl.Int64).round(1)  # Round all numeric columns to 1 decimal
    )
)

### Practice 3

Compute the body mass index for `masculine` characters who are `Human`.

- select only the columns you need
- filter out only the rows you need
- make the new variable with the body mass index
- sort the DataFrame starting with the highest body mass index

In [ ]:
# write your code here

## Additional resources

### Cheatsheets

- [Polars cheat sheet](https://franzdiebold.github.io/polars-cheat-sheet/Polars_cheat_sheet.pdf) --> summary of Polars operations
- [Python basics](https://ehmatthes.github.io/pcc_2e/cheat_sheets/cheat_sheets/) --> basic Python syntax and concepts

### Online courses

- [Polars documentation](https://pola.rs/py-polars/html/reference/) --> comprehensive Polars reference
- [Python for Data Analysis](https://wesmckinney.com/book/) --> comprehensive book on Python data analysis
- [Real Python](https://realpython.com/) --> practical Python tutorials
- [Kaggle Learn Python](https://www.kaggle.com/learn/python) --> free Python course

### Books and resources

- [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) --> comprehensive guide to Python data science
- [Polars User Guide](https://pola.rs/user-guide/) --> official Polars documentation and tutorials

## Session info

Information about this Python session including which version of Python was used, and what packages were loaded.

In [ ]:
import sys
import platform

print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")

# Package versions
packages = ['polars', 'seaborn', 'matplotlib', 'numpy']
for package in packages:
    try:
        exec(f"import {package}")
        version = eval(f"{package}.__version__")
        print(f"{package} version: {version}")
    except:
        print(f"{package}: version not available")